0. Recap Data

In [22]:
import nilearn
from nilearn import datasets
import pandas as pd
from glob import glob
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

In [7]:
# Load the open data set ADHD 200 from nilearn
data = nilearn.datasets.fetch_adhd(n_subjects=40, data_dir='/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data', url=None, resume=True, verbose=1)

In [13]:
data['func'][0]

'/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz'

In [16]:
data.keys()

dict_keys(['func', 'confounds', 'phenotypic', 'description'])

In [17]:
data['confounds'][0]

'/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/adhd/data/0010042/0010042_regressors.csv'

In [18]:
data_file_pheno = glob('data/adhd/ADHD200_40subs_motion*.csv')
rest_files = data['func'][0]
confound_files = data['confounds'][0]

In [19]:
# Path to Yeo atlas
atlas_yeo = datasets.fetch_atlas_yeo_2011(data_dir='/mnt/c/Users/nasta/Desktop/Forschungsmodul/project_msc05/data/yeo_atlas')

In [20]:
atlas_file = atlas_yeo.thick_7

In [25]:
masker = NiftiLabelsMasker(labels_img=atlas_file, standardize=True, memory='nilearn_cache')

In [28]:
time_series = masker.fit_transform(rest_files, confounds=confound_files)

In [29]:
time_series.shape

(176, 7)

In [30]:
connectome_measure = ConnectivityMeasure(kind='correlation')
correlation_matrices = connectome_measure.fit_transform(time_series)

ValueError: Each subject must be 2D numpy.ndarray.
 Youprovided arrays of dimensions [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]